In [32]:
# Import packages
import numpy as np
import pandas as pd
import datetime
from math import sqrt
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import QuantileRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [33]:
# Load data
data = pd.read_csv('..\\data\\shelter_neighbourhood_features_pca.csv')

In [34]:
# Drop columns that are not useful
df = data.drop(
    columns=[
        "_id",
        "ORGANIZATION_NAME",
        "SHELTER_GROUP",
        "LOCATION_NAME",
        "LOCATION_ADDRESS",
        "LOCATION_POSTAL_CODE",
        "LOCATION_CITY",
        "LOCATION_PROVINCE",
        "PROGRAM_NAME",
        "CAPACITY_ACTUAL_BED",
        "CAPACITY_FUNDING_BED",
        "OCCUPIED_BEDS",
        "UNOCCUPIED_BEDS",
        "UNAVAILABLE_BEDS",
        "CAPACITY_TYPE",
        "CAPACITY_ACTUAL_ROOM",
        "CAPACITY_FUNDING_ROOM",
        "OCCUPIED_ROOMS",
        "UNOCCUPIED_ROOMS",
        "UNAVAILABLE_ROOMS",
        "OCCUPANCY_RATE_BEDS",
        "OCCUPANCY_RATE_ROOMS",
        "Neighbourhood"
    ]
)

In [35]:
# Process date column
df["OCCUPANCY_DATE"] = pd.to_datetime(df["OCCUPANCY_DATE"])
df["month"] = df["OCCUPANCY_DATE"].dt.month
df["year"] = df["OCCUPANCY_DATE"].dt.year
df["day"] = df["OCCUPANCY_DATE"].dt.day

In [36]:
def get_previous_day_service_users():
    # adds previous day's service user count as new column to the data
    df["PREV_DATE"] = df["OCCUPANCY_DATE"] - datetime.timedelta(days=1)
    prev_day_df = df[[
        "OCCUPANCY_DATE", "ORGANIZATION_ID", "SHELTER_ID", 
        "LOCATION_ID", "PROGRAM_ID", "SECTOR", "PROGRAM_MODEL",
        "OVERNIGHT_SERVICE_TYPE", "PROGRAM_AREA", "SERVICE_USER_COUNT"
    ]].copy()
    
    prev_day_df.rename(columns={"OCCUPANCY_DATE": "PREV_DATE", "SERVICE_USER_COUNT": "PREV_SERVICE_USER_COUNT"}, inplace=True)
    
    result = pd.merge(df, prev_day_df, how="left",
                      on=["PREV_DATE",
                          "ORGANIZATION_ID",
                          "SHELTER_ID", 
                          "LOCATION_ID",
                          "PROGRAM_ID",
                          "SECTOR",
                          "PROGRAM_MODEL",
                          "OVERNIGHT_SERVICE_TYPE",
                          "PROGRAM_AREA"
                         ]
                     )
    return result

In [37]:
result = get_previous_day_service_users()
# check that PREV_DATE and PREV_SERVICE_USER_COUNT were added correctly
result[result["LOCATION_ID"] == 1024][["OCCUPANCY_DATE", "PREV_DATE", "SERVICE_USER_COUNT", "PREV_SERVICE_USER_COUNT"]].head(10)

,OCCUPANCY_DATE,PREV_DATE,SERVICE_USER_COUNT,PREV_SERVICE_USER_COUNT
58,2021-01-01,2020-12-31,13,NaN
193,2021-01-02,2021-01-01,13,13.0
328,2021-01-03,2021-01-02,14,13.0
463,2021-01-04,2021-01-03,15,14.0
599,2021-01-05,2021-01-04,16,15.0
735,2021-01-06,2021-01-05,15,16.0
871,2021-01-07,2021-01-06,16,15.0
1006,2021-01-08,2021-01-07,16,16.0
1141,2021-01-09,2021-01-08,16,16.0
1276,2021-01-10,2021-01-09,16,16.0


In [38]:
df[df.columns[:20]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133193 entries, 0 to 133192
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   OCCUPANCY_DATE          133193 non-null  datetime64[ns]
 1   ORGANIZATION_ID         133193 non-null  int64         
 2   SHELTER_ID              133193 non-null  int64         
 3   LOCATION_ID             133193 non-null  int64         
 4   PROGRAM_ID              133193 non-null  int64         
 5   SECTOR                  133193 non-null  object        
 6   PROGRAM_MODEL           133193 non-null  object        
 7   OVERNIGHT_SERVICE_TYPE  133193 non-null  object        
 8   PROGRAM_AREA            133193 non-null  object        
 9   SERVICE_USER_COUNT      133193 non-null  int64         
 10  LAT                     133193 non-null  float64       
 11  LON                     133193 non-null  float64       
 12  Neighbourhood Number    133193

In [39]:
df = result.copy()
df

,OCCUPANCY_DATE,ORGANIZATION_ID,SHELTER_ID,LOCATION_ID,PROGRAM_ID,SECTOR,PROGRAM_MODEL,OVERNIGHT_SERVICE_TYPE,PROGRAM_AREA,SERVICE_USER_COUNT,...,V114,V115,V116,V117,V118,month,year,day,PREV_DATE,PREV_SERVICE_USER_COUNT
0,2021-01-01,24,40,1103,15371,Families,Emergency,Motel/Hotel Shelter,COVID-19 Response,74,...,1.226938,-0.366075,-0.626308,-1.042742,0.777811,1,2021,1,2020-12-31,NaN
1,2021-01-01,24,40,1103,16211,Mixed Adult,Emergency,Motel/Hotel Shelter,COVID-19 Response,3,...,1.226938,-0.366075,-0.626308,-1.042742,0.777811,1,2021,1,2020-12-31,NaN
2,2021-01-01,24,40,1103,16192,Men,Emergency,Motel/Hotel Shelter,COVID-19 Response,24,...,1.226938,-0.366075,-0.626308,-1.042742,0.777811,1,2021,1,2020-12-31,NaN
3,2021-01-01,24,40,1103,16191,Mixed Adult,Emergency,Motel/Hotel Shelter,COVID-19 Response,25,...,1.226938,-0.366075,-0.626308,-1.042742,0.777811,1,2021,1,2020-12-31,NaN
4,2021-01-01,24,40,1103,16193,Women,Emergency,Motel/Hotel Shelter,COVID-19 Response,13,...,1.226938,-0.366075,-0.626308,-1.042742,0.777811,1,2021,1,2020-12-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133188,2023-10-11,17,78,1129,14671,Youth,Emergency,Shelter,Base Shelter and Overnight Services System,27,...,-0.137304,-0.341594,0.011689,0.865199,0.837428,10,2023,11,2023-10-10,27.0
133189,2023-10-11,31,52,1064,12292,Youth,Emergency,Shelter,Base Shelter and Overnight Services System,33,...,-0.241660,0.099253,-0.035334,-0.076425,0.154665,10,2023,11,2023-10-10,33.0
133190,2023-10-11,31,52,1064,12291,Youth,Transitional,Shelter,Base Shelter and Overnight Services System,17,...,-0.241660,0.099253,-0.035334,-0.076425,0.154665,10,2023,11,2023-10-10,17.0
133191,2023-10-11,38,81,1147,14891,Youth,Emergency,Shelter,Base Shelter and Overnight Services System,10,...,-0.614945,0.082707,-0.388099,0.522711,0.088628,10,2023,11,2023-10-10,10.0


In [40]:
# Encode categorical variables
le1 = LabelEncoder()
df["SECTOR"] = le1.fit_transform(df["SECTOR"])

le2 = LabelEncoder()
df["PROGRAM_MODEL"] = le2.fit_transform(df["PROGRAM_MODEL"])

le3 = LabelEncoder()
df["OVERNIGHT_SERVICE_TYPE"] = le3.fit_transform(df["OVERNIGHT_SERVICE_TYPE"])

le4 = LabelEncoder()
df["PROGRAM_AREA"] = le4.fit_transform(df["PROGRAM_AREA"])

In [41]:
# drop days with missing previous day user counts
og_num_obs = len(df)
df = df.dropna()
print("Number of rows dropped: ", og_num_obs - len(df))
# check that 2021-01-01 day was dropped
assert(len(df[(df["day"] == 1) & (df["month"] == 1) & (df["year"] == 2021)]) == 0)

Number of rows dropped:  342


In [42]:
# Split the DataFrame into training and testing sets
train_df = df[df['year'].isin([2021, 2022])]
train_df = train_df.drop(columns=["OCCUPANCY_DATE", "PREV_DATE"])
test_df = df[df['year'] == 2023]

In [43]:
# Split into features and target
x_train = train_df.drop("SERVICE_USER_COUNT", axis=1)
y_train = train_df["SERVICE_USER_COUNT"]

x_test = test_df.drop("SERVICE_USER_COUNT", axis=1)
y_test = test_df["SERVICE_USER_COUNT"]

In [44]:
# Train Random Forest
rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
rf.fit(x_train, y_train)

RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42)

In [45]:
# Test model
# Recursive Multi-step Forecast
total_y_pred = np.empty((0,))
total_y_test = test_df[test_df["OCCUPANCY_DATE"] == test_df["OCCUPANCY_DATE"].unique()[0]]["SERVICE_USER_COUNT"]
total_x_test = test_df[test_df["OCCUPANCY_DATE"] == test_df["OCCUPANCY_DATE"].unique()[0]].drop(columns=["SERVICE_USER_COUNT"])

for d in test_df["OCCUPANCY_DATE"].unique():
    # Predict one day at a time
    x_test = total_x_test[total_x_test["OCCUPANCY_DATE"] == d]
    x_test = x_test.drop(columns=["OCCUPANCY_DATE", "PREV_DATE"])
    
    y_pred = rf.predict(x_test)
    
    total_y_pred = np.concatenate((total_y_pred, y_pred))
    
    next_day_rows = test_df[test_df["OCCUPANCY_DATE"] == d + datetime.timedelta(days=1)].drop(columns=["PREV_SERVICE_USER_COUNT"])
    if len(next_day_rows) > 0:
        temp = total_x_test[total_x_test["OCCUPANCY_DATE"] == d][[
            "ORGANIZATION_ID",
              "SHELTER_ID", 
              "LOCATION_ID",
              "PROGRAM_ID",
              "SECTOR",
              "PROGRAM_MODEL",
              "OVERNIGHT_SERVICE_TYPE",
              "PROGRAM_AREA"
        ]].copy()
        temp["PREV_DATE"] = d
        temp["PREV_SERVICE_USER_COUNT"] = np.round(y_pred)

        result = pd.merge(next_day_rows, temp, how="left",
                      on=["PREV_DATE",
                          "ORGANIZATION_ID",
                          "SHELTER_ID", 
                          "LOCATION_ID",
                          "PROGRAM_ID",
                          "SECTOR",
                          "PROGRAM_MODEL",
                          "OVERNIGHT_SERVICE_TYPE",
                          "PROGRAM_AREA"
                         ]
                     )
        result  = result.dropna()
        total_y_test = np.concatenate((total_y_test, result["SERVICE_USER_COUNT"]))
        result = result.drop(columns=["SERVICE_USER_COUNT"])
        
        total_x_test = pd.concat([total_x_test, result], ignore_index=True, axis=0)

In [46]:
# Check model accuracy
mse = mean_squared_error(total_y_test, total_y_pred)
rmse = sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

mae = mean_absolute_error(total_y_test, total_y_pred)
print(f'Mean Absolute Error: {mae}')

Root Mean Squared Error: 15.117637622367278
Mean Absolute Error: 6.75578487961969


In [47]:
# Preview predicted and actual values
comparison_df = pd.DataFrame({'Predicted': total_y_pred, 'Actual': total_y_test})
print(comparison_df)

       Predicted  Actual
0        565.528     614
1        111.778     111
2        251.756     249
3        470.962     462
4        148.944     147
...          ...     ...
32600     20.000      27
32601     22.788      33
32602     11.986      17
32603     10.786      10
32604     42.800      34

[32605 rows x 2 columns]


In [49]:
# Save shelter predictions
final_result = total_x_test.copy()
final_result["SERVICE_USER_COUNT_PRED"] = total_y_pred

# Join back relevant columns
temp = test_df[[
    "OCCUPANCY_DATE",
    "ORGANIZATION_ID",
    "SHELTER_ID", 
    "LOCATION_ID",
    "PROGRAM_ID",
    "SECTOR",
    "PROGRAM_MODEL",
    "OVERNIGHT_SERVICE_TYPE",
    "PROGRAM_AREA",
    "SERVICE_USER_COUNT"
]].copy()

result = pd.merge(final_result, temp, how="left",
                      on=["OCCUPANCY_DATE",
                          "ORGANIZATION_ID",
                          "SHELTER_ID", 
                          "LOCATION_ID",
                          "PROGRAM_ID",
                          "SECTOR",
                          "PROGRAM_MODEL",
                          "OVERNIGHT_SERVICE_TYPE",
                          "PROGRAM_AREA"
                         ]
                     )

neighbourhoods = data[["Neighbourhood", "Neighbourhood Number"]].drop_duplicates()
result = pd.merge(result, neighbourhoods, how="left", on="Neighbourhood Number")

# Save predicted counts for shelters
result[["LOCATION_ID", "year", "month", "day", "LAT", "LON", "SERVICE_USER_COUNT", "SERVICE_USER_COUNT_PRED"]].to_csv("..\\outputs\\shelter_rf.csv", index=False)

# Save predicted counts for neighbourhoods
result[["Neighbourhood", "year", "month", "day", "LAT", "LON", "SERVICE_USER_COUNT", "SERVICE_USER_COUNT_PRED"]].to_csv("..\\outputs\\neighbourhood_rf.csv", index=False)